## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

1. [Знакомство с файлом](#Знакомство)
2. [Предобработка данных](#предобработка)  
    2.1. [Обработка пропусков](#пропуски)  
    2.2. [Замена типа данных](#заменатипа)  
    2.3. [Обработка дубликатов](#дубли)  
    2.4. [Лемматизация](#лемма)  
    2.5. [Категоризация данных](#категоризация)
3. [Есть ли зависимость от кол-ва детей?](#дети)  
4. [Есть ли зависимость от семейного положения?](#семья)  
5. [Есть ли зависимость от дохода?](#доход)  
6. [Влияет ли цель кредита?](#цель)  
7. [Общий вывод](#вывод)

# 1. Знакомство с данными <a name="Знакомство"></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В изучаемых данных присутствуют столбцы:
 - кол-во детей
 - общий стаж
 - возраст
 - образование
 - семейное положение
 - пол
 - тип занятости
 - наличие долга
 - общий доход
 - цель кредита  
 
1. В таблице 21525 строк, в стобцах "общий стаж" и "доход" есть равное кол-во пропущенных значений. Вероятно, это как-то связано. 
2. В столбце "общий стаж" присутствуют отрицательные значения, что, скорее всего, является ошибкой, т.к. общий стаж не может быть отрицательным. Также значения в этом столбце слишком большие для обозначения дней. Вероятно, ошибка при выгрузке. 
3. В столбце "дети" есть отрицательные значения, что противоречит логике, а также аномальное максимальное значение - 20. 
4. В столбце "возраст" есть нулевые значения, что также не может быть правдой.
5. В столбце "образование" есть строки с разным регистром

В ходе исследования планируется:
- обработать пустые/пропущенные/нулевые значения
- категоризировать данные
- обработать дубликаты
- унифицировать цели кредита
- найти зависимость между платежеспособностью клиентов и наличием детей/семейным положением/доходом/целью кредита


# Шаг 2. Предобработка данных <a name="предобработка"></a>

## 2.1. Обработка пропусков <a name="пропуски"></a>

In [6]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

### В нашей таблице 2174 пропуска в двух столбцах. Возможно, доход и стаж не указан у безработных. Проверим эту гипотезу

In [7]:
df.income_type.value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [8]:
df_null = df[(df['total_income'].isnull()) | (df['days_employed'].isnull())]

In [9]:
df_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


In [10]:
df_null.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [11]:
df_null.income_type.value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

### Гипотеза о том, что доход не указан только у безработных не подтвердилась. Но все 2174 строки содержат пропуски в обоих столбцах. Возможно, данные просто не выгрузились, либо не были заполнены изначально. 

### Замена всех 2174 пропусков на одинаковое значение может повлечь сильное статистическое отклонение. У нас несколько категорий занятости, соответственно, в каждой категории разный доход. Заменим пропущенные значения дохода на медианные по группам, чтобы минимизировать влияние на конечный результат. 

In [12]:
df.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [13]:
df['total_income'] = df.total_income.fillna(df.groupby('income_type')['total_income'].transform('median'))

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Проверим, сколько отрицательных значений в стоблце "дети"

In [15]:
df.children.value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

### Значений -1 в общей выборке не много, возможно, это ошибка при выгрузке, либо при заполнении. Добавим эти значения к тем строкам, где кол-во детей = 1

In [16]:
df['children'] = df.children.apply(abs)

In [17]:
df.children.value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

### Далее посмотрим, сколько нулевых значений в столбце с возрастом

In [18]:
df.dob_years.value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

### 101 запись с возрастом, равным 0

In [19]:
df.dob_years.mean()

43.29337979094077

In [20]:
df.loc[df.dob_years == 0]['income_type'].value_counts()

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64

In [21]:
df = df.drop(df[df.dob_years == 0].index)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
children            21424 non-null int64
days_employed       19260 non-null float64
dob_years           21424 non-null int64
education           21424 non-null object
education_id        21424 non-null int64
family_status       21424 non-null object
family_status_id    21424 non-null int64
gender              21424 non-null object
income_type         21424 non-null object
debt                21424 non-null int64
total_income        21424 non-null float64
purpose             21424 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Нулевых значений оказалось не много относительно общей выборки, было принято решение удалить строки, в которых возраст был равен 0

In [23]:
df['education'] = df.education.str.lower()

In [24]:
df.sample(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10935,0,-617.921271,44,среднее,1,женат / замужем,0,M,компаньон,0,181126.065104,операции со своей недвижимостью
3882,1,-1617.008140,27,высшее,0,Не женат / не замужем,4,F,сотрудник,0,67647.113484,покупка жилой недвижимости
4680,1,-4287.735813,53,среднее,1,в разводе,3,M,сотрудник,0,152125.834707,строительство жилой недвижимости
18707,0,-5333.122819,37,среднее,1,Не женат / не замужем,4,M,сотрудник,0,236472.657415,операции с недвижимостью
7271,0,-1246.749979,47,высшее,0,женат / замужем,0,F,госслужащий,0,114689.053738,сделка с подержанным автомобилем
21498,1,-1330.627998,32,среднее,1,гражданский брак,1,M,сотрудник,0,240767.575887,сыграть свадьбу
19611,0,-402.188945,37,начальное,3,в разводе,3,M,сотрудник,0,72830.324027,ремонт жилью
20849,0,-4591.706053,41,среднее,1,Не женат / не замужем,4,F,сотрудник,0,158468.575187,профильное образование
3839,1,NaN,42,высшее,0,гражданский брак,1,F,компаньон,0,172357.950966,автомобиль
104,0,366323.649429,59,среднее,1,Не женат / не замужем,4,F,пенсионер,0,163581.950073,покупка жилья для сдачи


### Привели столбец с образованием к нижнему регистру

### Проверим, какие вообще присутствуют значения в других столбцах

In [25]:
df.gender.value_counts()

F      14164
M       7259
XNA        1
Name: gender, dtype: int64

In [26]:
df = df.drop(df[df.gender == 'XNA'].index)

In [27]:
df.gender.value_counts()

F    14164
M     7259
Name: gender, dtype: int64

### В столбце с полом было неопределенное значение, удалили.

In [28]:
df.family_status.value_counts()

женат / замужем          12331
гражданский брак          4155
Не женат / не замужем     2797
в разводе                 1185
вдовец / вдова             955
Name: family_status, dtype: int64

### Вывод

- Пропущенные значения дохода были заменены на медианные, в зависимости от типа занятости
- Отрицательные значения были исправлены 
- Строки с 0 в графе "возраст" были удалены
- Образование приведено к единому нижнему регистру
- Строка с незаполенной графой "пол" удалена

## 2.2 Замена типа данных <a name="заменатипа"></a>

#### Единственное, что хотелось бы сделать, это изменить вещественный тип данных на целочисленный в столбце дохода для лучшей читабельности.

In [29]:
df['total_income'] = df.total_income.astype('int64')

In [30]:
df.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income          int64
purpose              object
dtype: object

In [31]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


## Обработка дубликатов <a name="дубли"></a>

In [32]:
df[df.duplicated(keep = False)].sort_values('dob_years').head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20297,1,NaN,23,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
8853,1,NaN,23,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
15892,0,NaN,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
19321,0,NaN,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
3452,0,NaN,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости
18328,0,NaN,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости
4216,0,NaN,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594,строительство жилой недвижимости
6312,0,NaN,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594,строительство жилой недвижимости
18349,1,NaN,30,высшее,0,женат / замужем,0,F,госслужащий,0,150447,покупка жилья для семьи
8629,1,NaN,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка коммерческой недвижимости


In [33]:
df.drop_duplicates(keep = 'first', inplace = True)

## 2.3 Лемматизация <a name="лемма"></a>

#### Посмотрим, на какие цели берут кредиты

In [34]:
df.purpose.value_counts()

свадьба                                   786
на проведение свадьбы                     764
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с коммерческой недвижимостью     648
операции с жильем                         646
покупка жилья                             640
жилье                                     640
покупка жилья для семьи                   637
строительство собственной недвижимости    633
недвижимость                              629
операции со своей недвижимостью           627
строительство жилой недвижимости          621
строительство недвижимости                619
покупка своего жилья                      619
покупка недвижимости                      617
ремонт жилью                              605
покупка жилой недвижимости                603
на покупку своего автомобиля              502
заняться высшим образованием      

### Одинаковые цели зачастую записаны по-разному. Нужно привести их к единому виду

In [35]:
from pymystem3 import Mystem 


In [36]:
m = Mystem() 

In [37]:
df['lemma'] = df.purpose.apply(m.lemmatize)

In [38]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


In [39]:
from collections import Counter 

In [40]:
Counter(m.lemmatize(' '.join(df['purpose'])))

Counter({'покупка': 5869,
         ' ': 54785,
         'жилье': 4436,
         'приобретение': 459,
         'автомобиль': 4284,
         'дополнительный': 902,
         'образование': 3995,
         'сыграть': 760,
         'свадьба': 2310,
         'операция': 2593,
         'с': 2904,
         'на': 2210,
         'проведение': 764,
         'для': 1286,
         'семья': 637,
         'недвижимость': 6327,
         'коммерческий': 1306,
         'жилой': 1224,
         'строительство': 1873,
         'собственный': 633,
         'подержать': 849,
         'свой': 2223,
         'со': 627,
         'заниматься': 900,
         'сделка': 938,
         'получение': 1309,
         'высокий': 1366,
         'подержанный': 113,
         'профильный': 435,
         'сдача': 649,
         'ремонт': 605,
         '\n': 1})

In [41]:
def purpose_short(lst):
    if 'автомобиль' in lst:
        return 'автомобиль'
    if 'образование' in lst:
        return 'образование'
    if 'свадьба' in lst:
        return 'свадьба'
    if 'ремонт' in lst:
        return 'ремонт'
    if 'жилье' in lst or 'недвижимость' in lst:
        return 'недвижимость'
    

In [42]:
df['lemma'] = df.lemma.apply(purpose_short)

In [43]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


### В основном встречаются кредиты на авто, на жилье, образование, ремонт и свадьбу, в соответствие с этим, значения были приведены к единому виду

## 2.4 Категоризация данных <a name="категоризация"></a>

#### Будем категоризировать уровень дохода. Так как у нас максимальная сумма небольшая, разобьем на категории по 50 тысяч

In [44]:
df['total_income_cat'] = pd.cut(df.total_income, [0, 100000, 150000, 200000, 250000, 300000, 350000, 500000, 1000000000],
    labels = ['низкий', 'ниже среднего', 'средний', 'выше среднего', 
              'высокий', 'очень высокий', 'неприлично высокий', 'дом на озере Комо'])

In [45]:
df.groupby('total_income_cat')['debt'].agg(['count', 'mean'])

,count,mean
total_income_cat,,
низкий,4440,0.079505
ниже среднего,7130,0.087097
средний,4742,0.084774
выше среднего,2240,0.073214
высокий,1321,0.065859
очень высокий,621,0.082126
неприлично высокий,636,0.064465
дом на озере Комо,222,0.063063


#### Также, разобьем возраст клиентов на категории

In [46]:
df3 = df[df.lemma == 'свадьба'].reset_index(drop = True)


In [47]:
def age_cat(age):
    if 18 <= age <= 30:
        return '18-30 лет'
    if 31 <= age <= 40:
        return '31-40 лет'
    if 41 <= age <= 50:
        return '41-50 лет'
    if age > 50:
        return '50 и более лет'

In [48]:
df3.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma,total_income_cat
0,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний
1,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,низкий
2,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,118514,сыграть свадьбу,свадьба,ниже среднего
3,0,-272.981385,21,высшее,0,гражданский брак,1,M,сотрудник,0,128265,сыграть свадьбу,свадьба,ниже среднего
4,0,-4649.910832,34,среднее,1,гражданский брак,1,F,сотрудник,1,139057,на проведение свадьбы,свадьба,ниже среднего


### Больше всего людей из выборки зарабатывают от 100 до 150 тысяч. Далее категории до 100 тысяч и от 150 до 200 тыс. 

# 3. Есть ли зависимость между наличием детей и возвратом кредита в срок? <a name="дети"></a>

In [49]:
df.groupby('children')['debt'].agg(['count', 'mean'])

,count,mean
children,,
0,14021,0.075458
1,4839,0.091341
2,2039,0.095145
3,328,0.082317
4,41,0.097561
5,9,0.000000
20,75,0.106667


### Меньше всего невозвратов наблюдается у заемщиков без детей. На 1,5%-2% больше невозвратов у семей с 1 или 2 детьми. 
### Кол-во людей с тремя и более детьми в нашей выборке относительно небольшое, поэтому выводы по этим категориям делать не совсем корректно, необходимо собрать больше информации. 

# 4. Есть ли зависимость между семейным положением и возвратом кредита в срок? <a name="семья"></a>

In [50]:
df.groupby('family_status')['debt'].agg(['count', 'mean'])

,count,mean
family_status,,
Не женат / не замужем,2794,0.097709
в разводе,1185,0.071730
вдовец / вдова,954,0.064990
гражданский брак,4129,0.093485
женат / замужем,12290,0.075427


#### Для полноты картины отобразим результаты в зависимости от пола

In [51]:
df.groupby(['family_status', 'gender'])['debt'].agg(['count', 'mean'])

count      mean
family_status         gender                 
Не женат / не замужем F        1721  0.068565
                      M        1073  0.144455
в разводе             F         927  0.065804
                      M         258  0.093023
вдовец / вдова        F         899  0.056730
                      M          55  0.200000
гражданский брак      F        2829  0.082008
                      M        1300  0.118462
женат / замужем       F        7726  0.068211
                      M        4564  0.087642

In [52]:
df.pivot_table(index = ['gender', 'family_status'], values = 'debt', aggfunc = ['count', 'mean'])

count      mean
                              debt      debt
gender family_status                        
F      Не женат / не замужем  1721  0.068565
       в разводе               927  0.065804
       вдовец / вдова          899  0.056730
       гражданский брак       2829  0.082008
       женат / замужем        7726  0.068211
M      Не женат / не замужем  1073  0.144455
       в разводе               258  0.093023
       вдовец / вдова           55  0.200000
       гражданский брак       1300  0.118462
       женат / замужем        4564  0.087642

In [53]:
df.pivot_table(index = ['gender'], values = 'debt', aggfunc = ['count', 'mean'])

,count,mean
,debt,debt
gender,,
F,14102,0.070132
M,7250,0.102621


#### 1) у женатых/замужних средний показатель невозвратов 7,5%, причем у женщин процент немного ниже, чем у мужчин
#### 2) у людей в гражданском браке показатель уже выше, 9,2%. И опять же, женщины возвращают долг чаще - 8,1% против 11,8% у мужчин. 
#### 3) у не женатых и не замужних показатель близок к гражданскому браку - 9,7%, но выборка в 2 раза меньше. И, судя по полученным данным, неженатые мужчины более халатно относятся к возврату долга - 14% невозвратов. 
#### 4) стоит обратить внимание на мужчин - вдовцов. Хоть и выборка крайне мала!!, 20% это неблагоприятный показатель. 

### В целом, женатые и разведенные люди возвращают долг чаще, чем не женатые / не замужние. Если говорить о девушках, то вне зависимости от их семейного положения, показатель примерно одинаковый - в районе 7%. Стоит обратить внимание на мужчин, находящихся в гражданском браке, либо не женатых. 

# 5. Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <a name="доход"></a>

In [54]:
df.groupby('total_income_cat')['debt'].agg(['count', 'mean'])

,count,mean
total_income_cat,,
низкий,4440,0.079505
ниже среднего,7130,0.087097
средний,4742,0.084774
выше среднего,2240,0.073214
высокий,1321,0.065859
очень высокий,621,0.082126
неприлично высокий,636,0.064465
дом на озере Комо,222,0.063063


#### Заодно добавим параметры образование и тип занятости:

In [55]:
df2 = df.groupby(['income_type', 'education', 'total_income_cat'])['debt'].agg(['mean', 'count'])

In [56]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 320 entries, (безработный, высшее, высокий) to (студент, ученая степень, средний)
Data columns (total 2 columns):
mean     127 non-null float64
count    127 non-null float64
dtypes: float64(2)
memory usage: 6.3+ KB


In [57]:
df2 = df2.dropna()

In [58]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 127 entries, (безработный, высшее, выше среднего) to (студент, высшее, низкий)
Data columns (total 2 columns):
mean     127 non-null float64
count    127 non-null float64
dtypes: float64(2)
memory usage: 2.7+ KB


#### 127 строк, среди которых есть выборки с маленьким кол-вом записей. Посмотрим наиболее вещественные:

In [59]:
df2[df2['count'] > 500]

mean   count
income_type education total_income_cat                  
компаньон   высшее    средний           0.068592   554.0
            среднее   ниже среднего     0.096525   777.0
                      средний           0.081897   928.0
пенсионер   среднее   ниже среднего     0.064935  1155.0
                      низкий            0.053114  1092.0
сотрудник   высшее    ниже среднего     0.058896   815.0
            среднее   выше среднего     0.094315   774.0
                      ниже среднего     0.106236  3191.0
                      низкий            0.100787  1905.0
                      средний           0.116264  1574.0

#### Люди с высоким достатком - от 350 тысяч в месяц - реже всего оказываются в должниках. Такой же низкий показатель (6,5%) и у людей с доходом  от 250 до 300 тысяч. Остальные категории варьируются от 8 до 8,6%. 
#### Если брать во внимание образование и тип занятости:
#### Наемные сотрудники со средним образованием имеют показатель 9,4-11,6%, в то время как с высшим образованием этот показатель снижается почти в 2 раза - 5,8%
#### Также низкий показатель невозвратов наблюдается у пенсионеров - 5,3 - 6,3 %

# 6. Как разные цели кредита влияют на его возврат в срок? <a name="цель"></a>

In [60]:
df.groupby('lemma')['debt'].agg(['mean', 'count'])

,mean,count
lemma,,
автомобиль,0.093371,4284
недвижимость,0.073243,10158
образование,0.092616,3995
ремонт,0.057851,605
свадьба,0.079654,2310


#### Посмотрим отдельно, в каком возрасте берут и как возвращают кредиты на свадьбу

In [61]:
df3['dob_years'] = df3.dob_years.apply(age_cat)

In [62]:
df3.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma,total_income_cat
0,0,340266.072047,50 и более лет,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний
1,2,-6929.865299,31-40 лет,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,низкий
2,0,NaN,50 и более лет,среднее,1,гражданский брак,1,M,пенсионер,0,118514,сыграть свадьбу,свадьба,ниже среднего
3,0,-272.981385,18-30 лет,высшее,0,гражданский брак,1,M,сотрудник,0,128265,сыграть свадьбу,свадьба,ниже среднего
4,0,-4649.910832,31-40 лет,среднее,1,гражданский брак,1,F,сотрудник,1,139057,на проведение свадьбы,свадьба,ниже среднего


In [63]:
df3.groupby(['lemma', 'dob_years'])['debt'].agg(['mean', 'count'])

mean  count
lemma   dob_years                      
свадьба 18-30 лет       0.096939    392
        31-40 лет       0.087637    639
        41-50 лет       0.085868    559
        50 и более лет  0.058333    720

In [64]:
df3.groupby(['lemma', 'family_status'])['debt'].agg(['mean', 'count'])

,,mean,count
lemma,family_status,,
свадьба,гражданский брак,0.079654,2310


### Самый низкий показатель в зависимости от цели кредита - у людей, занимающих на ремонт. Возможно, это связано с тем, что суммы на ремонт относительно небольшие, и их легче вернуть. А вот такие цели, как автомобиль и образование, имеют самый высокий показатель - выше 9%
### Люди младше 30 лет на 1% реже возвращают кредиты на свадьбу в срок. Как ни странно, наиболее добросовестные люди играют свадьбу после 50 лет :) 

# 7. Общий вывод <a name="вывод"></a>

#### Для исследования платежеспособности клиентов банка было проанализировано **21525** записей, содержащих информацию о кол-ве детей, стаже, возрасте, доходе, типе занятости, цели кредита и о своевременности возврата кредита.  Перед началом исследования данные были обработаны и подготовлены для обеспечения наиболее качественного анализа. В том числе:
 - Удалена **101** строка (из-за отсутствия информации о возрасте)
 - В **2174** строках пропущенные значения о доходе заменены на медианные значения в соответствии с типом занятости клиента
 - Записи были разбиты на категории в зависимости от возраста и уровня дохода
 - Цели кредита были приведены к единому стандарту

#### В ходе исследования выяснилось, что:
 - Самыми платежеспособными оказались:
   - люди без детей (7,5% невозвратов против 9-9,5% у семей с 1-2 детьми); 
   - женатые/замужние (7,5%);
   - с уровнем дохода от 250 тысяч (6,5%)
   - сотрудники с высшим образованием и пенсионеры (5,8%)
   - люди, взявшие кредит на ремонт (5,7%)
#### С ростом кол-ва детей наблюдается снижение платежеспособности, т.к. расходы на семью увеличиваются
#### Кредиты на авто и образование также имеют более низкий показатель возврата
#### Мужчины, состоящие в гражанском браке, либо не женатые, также оказываются менее платежеспособными, в отличие от женщин

#### Рекомендации:
 - Необходимо наладить выгрузку данных по параметру "общий трудовой стаж"
 - Выяснить, почему возникают пропуски в графах "доход" и "стаж"
 - проконтролировать заполнение графы "возраст"
 - проанализировать бОльшую выборку людей с 3 и более детьми
 - при выдаче кредитов отдавать предпочтение женатым/замужним, без детей, с уровнем дохода выше среднего
 - внимательно относиться к клиентам - мужчинам, т.к. согласно анализу предоставленной выборки, их платежеспособность зачастую оказывается ниже, чем у женщин, вне зависимости от дохода и семейного положения